# Explainer utility in BPMN2CONSTRAINTS

In this notebook, we explore the `Explainer` class, designed to analyze and explain the conformance of traces against predefined constraints. Trace analysis is crucial in domains such as process mining, where understanding the behavior of system executions against expected models can uncover inefficiencies, deviations, or compliance issues.

The constraints currently consists of basic regex, this is because of it's similiarities and likeness to declarative constraints used in BPMN2CONSTRAINTS


## Step 1: Setup

In [7]:
import sys
sys.path.append('../')
from explainer import Explainer, Trace

## Step 2: Basic Usage
Let's start by creating an instance of the `Explainer` and adding a simple constraint that a valid trace should contain the sequence "A" followed by "B" and then "C".


In [8]:
explainer = Explainer()
explainer.add_constraint('A.*B.*C')

## Step 3: Analyzing Trace Conformance

Now, we'll create a trace and check if it conforms to the constraints we've defined.

In [9]:
trace = Trace(['A', 'X', 'B', 'Y', 'C'])
is_conformant = explainer.conformant(trace)
print(f"Is the trace conformant? {is_conformant}")

Is the trace conformant? True


## Step 4: Explaining Non-conformance

If a trace is not conformant, we can use the `minimal_expl` and `counterfactual_expl` methods to understand why and how to adjust the trace.


In [10]:
non_conformant_trace = Trace(['A', 'C'])
print('Constraint: A.*B.*C')
print('Trace:' + str(non_conformant_trace.nodes))
print(explainer.counterfactual_expl(non_conformant_trace))
print(explainer.minimal_expl(non_conformant_trace))

non_conformant_trace = Trace(['C', 'B', 'A'])
print('-----------')
print('Constraint: A.*B.*C')
print('Trace:' + str(non_conformant_trace.nodes))
print(explainer.counterfactual_expl(non_conformant_trace))
print(explainer.minimal_expl(non_conformant_trace))

non_conformant_trace = Trace(['A','A','C'])
print('-----------')
print('Constraint: A.*B.*C')
print('Trace:' + str(non_conformant_trace.nodes))
print(explainer.counterfactual_expl(non_conformant_trace))
print(explainer.minimal_expl(non_conformant_trace))


non_conformant_trace = Trace(['A','A','C','A','TEST','A','C', 'X', 'Y']) 
print('-----------')
print('Constraint: A.*B.*C')
print('Trace:' + str(non_conformant_trace.nodes))
#print(explainer.counterfactual_expl(non_conformant_trace))
#print(explainer.minimal_expl(non_conformant_trace))


explainer.remove_constraint(0)
explainer.add_constraint('AC')
non_conformant_trace = Trace(['A', 'X', 'C']) #Substraction
print('-----------')
print('Constraint: AC')
print('Trace:' + str(non_conformant_trace.nodes))
print(explainer.counterfactual_expl(non_conformant_trace))
print(explainer.minimal_expl(non_conformant_trace))
print('-----------')

explainer.add_constraint('B.*A.*B.*C')
explainer.add_constraint('A.*B.*C.*')
explainer.add_constraint('A.*D.*B*')
explainer.add_constraint('A[^D]*B')
explainer.add_constraint('B.*[^X].*')
non_conformant_trace = Trace(['A', 'X', 'C']) #Substraction
for con in explainer.constraints:
    print(f'constraint: {con}')
print('Trace:' + str(non_conformant_trace.nodes))
print(explainer.counterfactual_expl(non_conformant_trace))
print(explainer.minimal_expl(non_conformant_trace))




Constraint: A.*B.*C
Trace:['A', 'C']

Addition (Added B at position 1): A->B->C
Non-conformance due to: Constraint (A.*B.*C) is violated by subtrace: ('A', 'C')
-----------
Constraint: A.*B.*C
Trace:['C', 'B', 'A']

Addition (Added A at position 1): C->A->B->A
Subtraction (Removed C from position 0): A->B->A
Addition (Added C at position 2): A->B->C->A
Non-conformance due to: Constraint (A.*B.*C) is violated by subtrace: ('C', 'B')
-----------
Constraint: A.*B.*C
Trace:['A', 'A', 'C']

Addition (Added B at position 2): A->A->B->C
Non-conformance due to: Constraint (A.*B.*C) is violated by subtrace: ('A', 'A')
-----------
Constraint: A.*B.*C
Trace:['A', 'A', 'C', 'A', 'TEST', 'A', 'C', 'X', 'Y']
-----------
Constraint: AC
Trace:['A', 'X', 'C']

Subtraction (Removed X from position 1): A->C
Non-conformance due to: Constraint (AC) is violated by subtrace: ('A', 'X')
-----------
constraint: AC
constraint: B.*A.*B.*C
constraint: A.*B.*C.*
constraint: A.*D.*B*
constraint: A[^D]*B
constraint:

## Step 5: Event Logs and Shapely values

The event logs in this context is built with traces, here's how you set them up.

In [11]:
from explainer import EventLog

event_log = EventLog()
trace1 = Trace(['A', 'B', 'C'])
trace2 = Trace(['B', 'C'])
trace3 = Trace(['A', 'B'])
trace4 = Trace(['B'])

event_log.add_trace(trace1, 5) # The second is how many traces you'd like to add, leave blank for 1
event_log.add_trace(trace2, 10)
event_log.add_trace(trace3, 5)
event_log.add_trace(trace4, 5)


exp = Explainer()
exp.add_constraint("C$")
exp.add_constraint("^A")
print("Conformance rate: "+ str(exp.determine_conformance_rate(event_log)))
print('Contribution C$:', exp.determine_shapley_value(event_log, exp.constraints, 0))
print('Contribution ^A:', exp.determine_shapley_value(event_log, exp.constraints, 1))


Conformance rate: 0.2
Contribution C$: -0.09999999999999998
Contribution ^A: 0.09999999999999998


In [12]:
exp = Explainer()
event_log = EventLog()
trace1 = Trace(['A', 'B', 'C'])
trace2 = Trace(['B', 'C'])
trace3 = Trace(['A', 'B'])
trace4 = Trace(['B'])
trace5 = Trace(['A', 'C'])


event_log.add_trace(trace1, 5) # The second is how many traces you'd like to add, leave blank for 1
event_log.add_trace(trace2, 10)
event_log.add_trace(trace3, 5)
event_log.add_trace(trace4, 5)
event_log.add_trace(trace5, 10)


exp = Explainer()
exp.add_constraint("C$")
exp.add_constraint("^A")
exp.add_constraint("B*\{1\}")
print("conformant AC :" + str(exp.conformant(trace5)))
print("Conformance rate: "+ str(exp.determine_conformance_rate(event_log)))
print('Contribution C$:', exp.determine_shapley_value(event_log, exp.constraints, 0))
print('Contribution ^A:', exp.determine_shapley_value(event_log, exp.constraints, 1))
print('Contribution B*:', exp.determine_shapley_value(event_log, exp.constraints, 2))



error: multiple repeat at position 2